In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm


In [24]:
df = pd.read_csv('./data/stock-market-data.csv')


In [36]:
col = df.columns


In [25]:
df[:4]


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,2.902296e+08,6.655275,1.255006e+11,0.046409,0.021259,0.0,BANKS
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1.144753e+09,6.655275,1.290820e+11,0.046409,0.021259,0.0,BANKS
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,5.370438e+08,6.655275,1.299774e+11,0.046409,0.021259,0.0,BANKS
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,7.114296e+08,6.655275,1.335589e+11,0.046409,0.021259,0.0,BANKS


In [26]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276900 entries, 0 to 276899
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   symbol      276900 non-null  object 
 1   date        276900 non-null  int64  
 2   pre_close   276900 non-null  float64
 3   open        276900 non-null  float64
 4   high        276900 non-null  float64
 5   low         276900 non-null  float64
 6   close       276900 non-null  float64
 7   volume      276900 non-null  int64  
 8   amount      276900 non-null  float64
 9   adj_factor  276900 non-null  float64
 10  capt        276900 non-null  float64
 11  index_w50   276900 non-null  float64
 12  index_w300  276900 non-null  float64
 13  index_w500  276900 non-null  float64
 14  industry    276900 non-null  object 
dtypes: float64(11), int64(2), object(2)
memory usage: 31.7+ MB


In [27]:
df[['symbol', 'industry']] = df[['symbol', 'industry']].apply(
    lambda x: x.astype("string"))


In [28]:
# 1
a1 = set(df['symbol'][df['symbol'].apply(lambda x: x.count('8') > 0)])


In [29]:
# 2
# df['Flag'] = np.where(df['close'] > df['pre_close'],'Increase',np.where(df['close'] < df['pre_close'],'Decrease','Flat'))

conditions = [
    df['close'] > df['pre_close'],
    df['close'] < df['pre_close']
]

results = ['Increase', 'Decrease']
df['Flag'] = np.select(conditions, results, default='Flat')

df.groupby(['date', 'Flag']).agg({'symbol': 'nunique'})


symbol
date     Flag            
20120104 Decrease    2007
         Flat         122
         Increase     191
20120105 Decrease    2071
         Flat         117
...                   ...
20120628 Flat         176
         Increase     427
20120629 Decrease     358
         Flat         174
         Increase    1890

[351 rows x 1 columns]

In [32]:
df['exchange'] = df['symbol'].str[-2:]
df.head()


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry,Flag,exchange
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,2.902296e+08,6.655275,1.255006e+11,0.046409,0.021259,0.0,BANKS,Decrease,SH
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1.144753e+09,6.655275,1.290820e+11,0.046409,0.021259,0.0,BANKS,Increase,SH
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,5.370438e+08,6.655275,1.299774e+11,0.046409,0.021259,0.0,BANKS,Increase,SH
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,7.114296e+08,6.655275,1.335589e+11,0.046409,0.021259,0.0,BANKS,Increase,SH
4,600000.SH,20120110,8.95,8.95,9.10,8.88,9.07,72004632,6.472066e+08,6.655275,1.353496e+11,0.046409,0.021259,0.0,BANKS,Increase,SH


In [45]:
# 3
df.groupby(['date', 'exchange', 'Flag']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,exchange,Flag,symbol
0,20120104,SH,Decrease,794
1,20120104,SH,Flat,42
2,20120104,SH,Increase,85
3,20120104,SZ,Decrease,1213
4,20120104,SZ,Flat,80


In [44]:
# 4

df.query("index_w300 > 0").groupby(['date', 'Flag']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,Flag,symbol
0,20120104,Decrease,275
1,20120104,Flat,5
2,20120104,Increase,20
3,20120105,Decrease,242
4,20120105,Flat,8


In [46]:
# 5
df.groupby(['date', 'industry']).agg({'symbol': 'nunique'}).reset_index()[:5]


,date,industry,symbol
0,20120104,AERODEF,10
1,20120104,AIRLINE,12
2,20120104,AUTO,85
3,20120104,BANKS,16
4,20120104,BEV,30


In [68]:
# 6
df1 = df.groupby(['date', 'industry']).agg(
    {'symbol': 'nunique', 'amount': 'sum'}).reset_index()


In [70]:
df1[['max_n', 'max_a']] = df1.groupby(
    ['date'])[['symbol', 'amount']].transform('max')


In [71]:
df1.query('symbol == max_n & amount == max_a')


,date,industry,symbol,amount,max_n,max_a
15,20120104,HDWRSEMI,224,6.878138e+09,224,6.878138e+09
79,20120106,HDWRSEMI,224,5.979259e+09,224,5.979259e+09
751,20120213,HDWRSEMI,227,1.460269e+10,227,1.460269e+10
847,20120216,HDWRSEMI,227,1.506463e+10,227,1.506463e+10
879,20120217,HDWRSEMI,229,1.145135e+10,229,1.145135e+10
911,20120220,HDWRSEMI,230,1.525181e+10,230,1.525181e+10
943,20120221,HDWRSEMI,230,1.549184e+10,230,1.549184e+10
975,20120222,HDWRSEMI,230,1.990051e+10,230,1.990051e+10
1167,20120301,HDWRSEMI,231,1.486525e+10,231,1.486525e+10
1199,20120302,HDWRSEMI,231,1.620855e+10,231,1.620855e+10


In [72]:
del df1


In [78]:
# 7

conditions = [
    (df['close'] - df['pre_close']) / df['pre_close'] > 0.05,
    (df['close'] - df['pre_close']) / df['pre_close'] < - 0.05
]

results = ['Increase 5%', 'Decrease 5%']
df['Flag%'] = np.select(conditions, results, default='Flat')

df.query('`Flag%` != "Flat"').groupby(['date', 'Flag%']).agg(
    {'symbol': 'nunique'}).reset_index()[:5]


,date,Flag%,symbol
0,20120104,Decrease 5%,277
1,20120104,Increase 5%,17
2,20120105,Decrease 5%,885
3,20120105,Increase 5%,10
4,20120106,Decrease 5%,66


In [86]:
# 8
df['rate%'] = (df['close'] - df['pre_close']) / df['pre_close']

df.sort_values(by='rate%', ascending=False, inplace=True)


In [89]:

df1 = df.groupby(['date']).agg(top10=('amount', lambda x: sum(x[:10])),
                               last10=('amount', lambda x: sum(x[-10:])))


In [96]:
pd.options.display.float_format = '{:.2f}'.format


In [104]:
pd.set_option("display.max_rows", 150)


In [105]:
df1['top/bottom'] = df1['top10']/df1['last10']
df1


,top10,last10,top/bottom
date,,,
20120104,660522056.10,456926638.10,1.45
20120105,660035624.90,383022701.40,1.72
20120106,947170129.80,703279193.90,1.35
20120109,1586198245.50,558059839.50,2.84
20120110,1658431604.10,2948836472.30,0.56
20120111,3024195777.40,2144084175.70,1.41
20120112,1215494588.20,2950479801.60,0.41
20120113,2611351394.70,875104112.30,2.98
20120116,458861322.50,625366458.30,0.73


In [110]:
# 9
df['ret_open'] = df['open']/df['pre_close'] > 1.015
df['ret_close'] = df['close']/df['pre_close'] > 1.015


In [ ]:
df.groupby('date')[['ret_open', 'ret_close']].sum().reset_index()


In [145]:
# 10
flag_open = []

for i in range(3, df.shape[0]-1):

    flag_open.append(
        sum(map(int, [df['ret_open'][i-3], df['ret_open'][i-2], df['ret_open'][i-1]])))


In [150]:
filled_open = [None] * 4 + flag_open


In [155]:
df['flag_open'] = pd.Series(filled_open)


In [157]:
df.groupby('date')[['flag_open']].sum()


,flag_open
date,
20120104,130.00
20120105,126.00
20120106,414.00
20120109,373.00
20120110,408.00
20120111,156.00
20120112,224.00
20120113,243.00
20120116,209.00


In [158]:
df.query('flag_open>0').head()


,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,...,index_w300,index_w500,industry,Flag,exchange,Flag%,rate%,ret_open,ret_close,flag_open
275374,300295.SZ,20120315,34.00,55.00,59.80,55.00,59.40,6821773,390459756.50,1.00,...,0.00,0.00,SOFTWARE,Increase,SZ,Increase 5%,0.75,True,True,1.00
275445,300296.SZ,20120315,16.00,22.20,24.98,22.18,24.98,12063503,286407311.30,1.00,...,0.00,0.00,HDWRSEMI,Increase,SZ,Increase 5%,0.56,True,True,1.00
275298,300294.SZ,20120308,25.00,38.00,39.00,37.01,38.80,9590069,364320477.70,1.00,...,0.00,0.00,HEALTH,Increase,SZ,Increase 5%,0.55,True,True,1.00
276896,300332.SZ,20120628,8.18,10.45,11.30,10.12,10.93,59788245,640027000.90,1.00,...,0.00,0.00,MACH,Increase,SZ,Increase 5%,0.34,True,True,1.00
112229,000036.SZ,20120420,3.85,3.80,4.24,3.79,4.24,137649204,568342939.30,6.97,...,0.00,0.00,REALEST,Increase,SZ,Increase 5%,0.10,False,True,2.00
